In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

## Combining dataframes

 Create some toy data

In [2]:
df1 = pd.DataFrame({'Name': ['Tom Cruise', 'Robert Downey Jr', 'Chris Hemsworth', 'Jennifer Lawrence'],\
                   'Age':[56, 53, 35, 28 ],\
                   'Gender': ['Male', 'Male', 'Male', 'Female']})

In [3]:
df1

,Name,Age,Gender
0,Tom Cruise,56,Male
1,Robert Downey Jr,53,Male
2,Chris Hemsworth,35,Male
3,Jennifer Lawrence,28,Female


In [4]:
df2 = pd.DataFrame({'Actor_name': ['Jennifer Lawrence','Tom Cruise', 'Johnny Depp', 'Kim Kardashian'],\
                  'Gender': ['Female', 'Male', 'Male', 'Female'],\
                  'Net_worth_millions': [60, 470, 400, 85]})

In [5]:
df2

,Actor_name,Gender,Net_worth_millions
0,Jennifer Lawrence,Female,60
1,Tom Cruise,Male,470
2,Johnny Depp,Male,400
3,Kim Kardashian,Female,85


Pandas `merge` allows us to join the two dataframes.

Documentation: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html

Join by the name?

In [6]:
df = pd.merge(df1, df2, left_on = 'Name', right_on = 'Actor_name') #left_on and right_on specifies the column names we want to join on 
df

,Name,Age,Gender_x,Actor_name,Gender_y,Net_worth_millions
0,Tom Cruise,56,Male,Tom Cruise,Male,470
1,Jennifer Lawrence,28,Female,Jennifer Lawrence,Female,60


Drop the duplicated columns?

In [7]:
df = df.drop(['Actor_name', 'Gender_y'],axis=1)
df

,Name,Age,Gender_x,Net_worth_millions
0,Tom Cruise,56,Male,470
1,Jennifer Lawrence,28,Female,60


The four types of `merge`

In [8]:
df = pd.merge(df1, df2, left_on = 'Name', right_on = 'Actor_name', how = 'left')
df

,Name,Age,Gender_x,Actor_name,Gender_y,Net_worth_millions
0,Tom Cruise,56,Male,Tom Cruise,Male,470.0
1,Robert Downey Jr,53,Male,NaN,NaN,NaN
2,Chris Hemsworth,35,Male,NaN,NaN,NaN
3,Jennifer Lawrence,28,Female,Jennifer Lawrence,Female,60.0


In [9]:
df = pd.merge(df1, df2, left_on = 'Name', right_on = 'Actor_name', how = 'right') 
df

,Name,Age,Gender_x,Actor_name,Gender_y,Net_worth_millions
0,Tom Cruise,56.0,Male,Tom Cruise,Male,470
1,Jennifer Lawrence,28.0,Female,Jennifer Lawrence,Female,60
2,NaN,NaN,NaN,Johnny Depp,Male,400
3,NaN,NaN,NaN,Kim Kardashian,Female,85


In [10]:
df = pd.merge(df1, df2, left_on = 'Name', right_on = 'Actor_name', how = 'outer') 
df

,Name,Age,Gender_x,Actor_name,Gender_y,Net_worth_millions
0,Tom Cruise,56.0,Male,Tom Cruise,Male,470.0
1,Robert Downey Jr,53.0,Male,NaN,NaN,NaN
2,Chris Hemsworth,35.0,Male,NaN,NaN,NaN
3,Jennifer Lawrence,28.0,Female,Jennifer Lawrence,Female,60.0
4,NaN,NaN,NaN,Johnny Depp,Male,400.0
5,NaN,NaN,NaN,Kim Kardashian,Female,85.0


In [11]:
df = pd.merge(df1, df2, left_on = 'Name', right_on = 'Actor_name', how = 'inner') 
df

,Age,Gender_x,Name,Actor_name,Gender_y,Net_worth_millions
0,56,Male,Tom Cruise,Tom Cruise,Male,470
1,28,Female,Jennifer Lawrence,Jennifer Lawrence,Female,60


As we can see, the default method is `inner`.

Since the dataframes have two common columns, we can also join on both.

In [11]:
df = pd.merge(df1, df2, left_on = ['Name', 'Gender'], right_on = ['Actor_name', 'Gender'])
df

,Name,Age,Gender,Actor_name,Net_worth_millions
0,Tom Cruise,56,Male,Tom Cruise,470
1,Jennifer Lawrence,28,Female,Jennifer Lawrence,60


### `merge` vs `concatenate`? (union)

In [12]:
df = pd.concat([df1, df2])
df

C:\Users\yyezeret\AppData\Local\Continuum\Anaconda2\envs\dviz\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Actor_name,Age,Gender,Name,Net_worth_millions
0,NaN,56.0,Male,Tom Cruise,NaN
1,NaN,53.0,Male,Robert Downey Jr,NaN
2,NaN,35.0,Male,Chris Hemsworth,NaN
3,NaN,28.0,Female,Jennifer Lawrence,NaN
0,Jennifer Lawrence,NaN,Female,NaN,60.0
1,Tom Cruise,NaN,Male,NaN,470.0
2,Johnny Depp,NaN,Male,NaN,400.0
3,Kim Kardashian,NaN,Female,NaN,85.0


`concat` simply sticks the dataframes together.

We can also stick horizontally.

In [13]:
df = pd.concat([df1, df2], axis = 'columns')
df

,Name,Age,Gender,Actor_name,Gender,Net_worth_millions
0,Tom Cruise,56,Male,Jennifer Lawrence,Female,60
1,Robert Downey Jr,53,Male,Tom Cruise,Male,470
2,Chris Hemsworth,35,Male,Johnny Depp,Male,400
3,Jennifer Lawrence,28,Female,Kim Kardashian,Female,85


## Grouping data

load a slightly larger dataset.

In [14]:
titanic = sns.load_dataset('titanic')

In [15]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


What if we want to look at the data grouped by the sex of passengers?

Documentation: http://pandas.pydata.org/pandas-docs/stable/groupby.html

In [24]:
grp = titanic.groupby('sex')
grp

In [25]:
grp.groups

{'female': Int64Index([  1,   2,   3,   8,   9,  10,  11,  14,  15,  18,
             ...
             866, 871, 874, 875, 879, 880, 882, 885, 887, 888],
            dtype='int64', length=314),
 'male': Int64Index([  0,   4,   5,   6,   7,  12,  13,  16,  17,  20,
             ...
             873, 876, 877, 878, 881, 883, 884, 886, 889, 890],
            dtype='int64', length=577)}

In [26]:
for name, group in grp:
    print(name)
    print(group)


female
     survived  pclass     sex   age  sibsp  parch      fare embarked   class  \
1           1       1  female  38.0      1      0   71.2833        C   First   
2           1       3  female  26.0      0      0    7.9250        S   Third   
3           1       1  female  35.0      1      0   53.1000        S   First   
8           1       3  female  27.0      0      2   11.1333        S   Third   
9           1       2  female  14.0      1      0   30.0708        C  Second   
10          1       3  female   4.0      1      1   16.7000        S   Third   
11          1       1  female  58.0      0      0   26.5500        S   First   
14          0       3  female  14.0      0      0    7.8542        S   Third   
15          1       2  female  55.0      0      0   16.0000        S  Second   
18          0       3  female  31.0      1      0   18.0000        S   Third   
19          1       3  female   NaN      0      0    7.2250        C   Third   
22          1       3  female  15

In [27]:
grp.get_group('female')

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False
10,1,3,female,4.0,1,1,16.7000,S,Third,child,False,G,Southampton,yes,False
11,1,1,female,58.0,0,0,26.5500,S,First,woman,False,C,Southampton,yes,True
14,0,3,female,14.0,0,0,7.8542,S,Third,child,False,NaN,Southampton,no,True
15,1,2,female,55.0,0,0,16.0000,S,Second,woman,False,NaN,Southampton,yes,True
18,0,3,female,31.0,1,0,18.0000,S,Third,woman,False,NaN,Southampton,no,False


In [28]:
grp.size()

sex
female    314
male      577
dtype: int64

In [29]:
grp.aggregate(np.mean)

,survived,pclass,age,sibsp,parch,fare,adult_male,alone
sex,,,,,,,,
female,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818,0.000000,0.401274
male,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893,0.930676,0.712305


Group by more than 1 columns?

In [0]:
grp = titanic.groupby(['sex', 'pclass'])

In [34]:
grp.groups

{('female',
  1): Int64Index([  1,   3,  11,  31,  52,  61,  88, 136, 151, 166, 177, 194, 195,
             215, 218, 230, 256, 257, 258, 268, 269, 275, 290, 291, 297, 299,
             306, 307, 309, 310, 311, 318, 319, 325, 329, 334, 337, 341, 356,
             366, 369, 375, 380, 383, 393, 412, 435, 457, 486, 496, 498, 504,
             513, 520, 523, 537, 539, 540, 556, 558, 571, 577, 581, 585, 591,
             609, 627, 641, 669, 689, 700, 708, 710, 716, 730, 742, 759, 763,
             765, 779, 781, 796, 809, 820, 829, 835, 842, 849, 853, 856, 862,
             871, 879, 887],
            dtype='int64'),
 ('female',
  2): Int64Index([  9,  15,  41,  43,  53,  56,  58,  66,  84,  98, 123, 133, 161,
             190, 199, 211, 237, 247, 259, 272, 303, 312, 316, 322, 323, 327,
             345, 346, 357, 387, 389, 399, 416, 417, 426, 427, 432, 437, 440,
             443, 446, 458, 472, 473, 506, 516, 518, 526, 530, 535, 546, 576,
             580, 596, 600, 608, 615, 618, 635, 651

Often after creating groups, we want to do some calculation for each group.

e.g. Calculate the average age of each sex?

In [0]:
grp = titanic.groupby('sex').mean()

In [42]:
grp

,survived,pclass,age,sibsp,parch,fare,adult_male,alone
sex,,,,,,,,
female,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818,0.000000,0.401274
male,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893,0.930676,0.712305


The group names are used as the new index.

Can be changed using the `reset_index` method

In [24]:
grp.reset_index()

,sex,survived,pclass,age,sibsp,parch,fare,adult_male,alone
0,female,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818,0.000000,0.401274
1,male,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893,0.930676,0.712305


Alternatively:

In [25]:
grp = titanic.groupby('sex').agg(np.mean)
grp.reset_index()

,sex,survived,pclass,age,sibsp,parch,fare,adult_male,alone
0,female,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818,0.000000,0.401274
1,male,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893,0.930676,0.712305


Applying multiple calculations

In [26]:
grp = titanic.groupby('sex')
grp['age'].agg([np.sum, np.mean, np.std]).reset_index()

,sex,sum,mean,std
0,female,7286.00,27.915709,14.110146
1,male,13919.17,30.726645,14.678201
